In [2]:
import json
import scipy.sparse as ss
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import scipy.io as sio

np.random.seed(123)

In [7]:
foil_train = json.load(open('data/foilv1.0_train2017.json'))
ourimagetrainfeats = [(int(w.strip().split(' ')[0].split('.jpg')[0].split('/data/mscoco/images/train/COCO_train2014_')[1]), 
                       np.array(map(float, w.strip().split(' ')[1:]))) 
                      for w in open('/data/mscoco_feats/ResNet152/ResNet152_pool5_train.txt')]
ourimagetrainfeats = dict(ourimagetrainfeats)
training_annotations = [l['caption'] for l in foil_train['annotations']]

tf_vectorizer = CountVectorizer(max_features=None, lowercase=True)
tf_model = tf_vectorizer.fit(training_annotations)
training_feats = tf_model.transform(training_annotations)

training_y = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_train['annotations']]
training_image_feats = [ourimagetrainfeats[i['image_id']] for i in foil_train['annotations']]
training_image_feats_sparse = np.array(training_image_feats)


default = testing_image_feats[0]
ourimagetestfeats = [(int(w.strip().split(' ')[0].split('.jpg')[0].split('/data/mscoco/images/val/COCO_val2014_')[1]),
                     map(float, w.strip().split(' ')[1:])) 
                    for w in open('/data/mscoco_feats/ResNet152/ResNet152_fc_val.txt')]
ourimagetestfeats = dict(ourimagetestfeats)
foil_test = json.load(open('data/foilv1.0_test2017.json') )
testing_annotations = [l['caption'] for l in foil_test['annotations']]
testing_feats = tf_model.transform(testing_annotations) 
testing_y = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_test['annotations']]
testing_image_feats = [ourimagetestfeats[i['image_id']] if i['image_id'] in ourimagetestfeats else default for i in foil_test['annotations']  ]
testing_image_feats_sparse = np.array(testing_image_feats)

In [9]:
#sio.mmwrite('data/train_feats.mtx', training_feats)
#sio.mmwrite('data/test_feats.mtx', testing_feats)
#sio.mmwrite('data/train_image_feats_resnet.mtx', training_image_feats_sparse)
#sio.mmwrite('data/test_image_feats_resnet.mtx', testing_image_feats_sparse)
#sio.mmwrite('data/train_y.mtx', ss.csr_matrix(np.array(training_y)))
#sio.mmwrite('data/test_y.mtx', ss.csr_matrix(np.array(testing_y)))

np.save('data/train_image_feats_resnet.npy', training_image_feats_sparse)
np.save('data/test_image_feats_resnet.npy', testing_image_feats_sparse)

In [8]:
len(testing_y)

150556

In [23]:
sum(testing_y)

75278

In [20]:
foil_train['annotations'][0]

{u'caption': u'A long restaurant table with rattan rounded back chairs.',
 u'foil_id': 2000001,
 u'foil_word': u'ORIG',
 u'id': 789366,
 u'image_id': 57870,
 u'target_word': u'ORIG'}

In [21]:
int(w.strip().split(' ')[0].split('.jpg')[0].split('/data/mscoco/images/train/COCO_train2014_')[1])

487025

In [31]:
len(w.strip().split(' ')[1:])

21265000